In [215]:
import numpy as np
import soundfile as sf

# import Vokaturi
import sys
sys.path.append("open_vok/api")
import Vokaturi
Vokaturi.load("open_vok/lib/Vokaturi_mac.so")

import audiotranscode
import emoji

from util import logger, emotion_file_path, emo_distribution, to_text, emodict_from_samples
from accessories import actions, rev_actions, rev_mapping

import tqdm

# Finding audio samples of emotions

In [200]:
def get_emo_dicts(file_name_in, nsec=10):
    samples, sample_rate = sf.read(file_name_in)
    
    def sec_to_ind(second):
        return sample_rate * second
    
    tmp_indices = np.arange(sec_to_ind(nsec), len(samples), sec_to_ind(nsec))

    splitted = np.split(samples, tmp_indices)[:-1]
    assert all(list(map(lambda x: len(x) == sec_to_ind(nsec), splitted)))
    
    emo_dicts = list(map(lambda x: emodict_from_samples(x, sample_rate), splitted))
    emo_dicts = list(filter(lambda x: x[0] == 1, emo_dicts))
    emo_dicts = list(map(lambda x: x[1], emo_dicts))
    emo_dicts = list(zip(emo_dicts, tmp_indices))
    
    return splitted, emo_dicts

def get_sorted(splitted, emo_dicts, emotion):
    emotions = list(map(lambda x: x[0][emotion], emo_dicts))
    emotions = list(enumerate(emotions))
    
    emotions_sorted = sorted(emotions, key=lambda x: x[1], reverse=True)
    
    return splitted, emotions_sorted

In [190]:
splitted, emo_dicts = get_emo_dicts('../tim_urban.wav')

In [216]:
for emotion in tqdm.tqdm(["neutrality", "happiness", "sadness", "anger", "fear"]):
    splitted, emotions_sorted = get_sorted(splitted, emo_dicts, emotion)
    topn = 4
    for record in emotions_sorted[:4]:
        file_name_out = 'res/audio_emotions/{}_{}.wav'.format(emotion, str(record[1])[2:])
        sample_to_save = splitted[record[0]]
        sf.write(file_name_out, sample_to_save, sample_rate, 'PCM_24')

100%|██████████| 5/5 [00:01<00:00,  4.38it/s]


# Sending audio samples of emotions

In [222]:
import os

def get_dict_of_emotions():
    """
    :return: dict of emotions and paths to audio examples
    """
    list_res = list(filter(lambda x: x[0] != '.', os.listdir(os.path.join('res', 'audio_emotions'))))
    splitted = list(map(lambda x: x.split('_'), list_res))
    tuples3 = list(map(lambda x: (x[1], '_'.join(x)), filter(lambda x: x[0] == '3', splitted)))

    return dict(tuples3)

In [223]:
get_dict_of_emotions()

{'anger': '3_anger_9995287435466227.wav',
 'fear': '3_fear_9755983599427229.wav',
 'happiness': '3_happiness_7852061984234702.wav',
 'neutrality': '3_neutrality_987074043745762.wav',
 'sadness': '3_sadness_08248565414639678.wav'}

In [234]:
list_res = list(filter(lambda x: x[0] != '.', os.listdir(os.path.join('res', 'audio_emotions'))))
splitted = list(map(lambda x: x.split('_'), list_res))
tuples2 = list(map(lambda x: (x[0], '_'.join(x)), filter(lambda x: len(x) == 2, splitted)))

In [235]:
tuples2

[('sadness', 'sadness_2703179208004744.wav'),
 ('fear', 'fear_7699375940356165.wav'),
 ('happiness', 'happiness_5944161044225376.wav'),
 ('fear', 'fear_8464279948590564.wav'),
 ('anger', 'anger_9992815572033784.wav'),
 ('neutrality', 'neutrality_987074043745762.wav'),
 ('happiness', 'happiness_542702566775099.wav'),
 ('fear', 'fear_9755983599427229.wav'),
 ('anger', 'anger_9995287435466227.wav'),
 ('happiness', 'happiness_6045050145985089.wav'),
 ('sadness', 'sadness_11544415614705221.wav'),
 ('sadness', 'sadness_2401697077223822.wav'),
 ('anger', 'anger_9994545645155642.wav'),
 ('happiness', 'happiness_7852061984234702.wav'),
 ('neutrality', 'neutrality_6988170650367929.wav'),
 ('fear', 'fear_9196001111935143.wav'),
 ('neutrality', 'neutrality_723136625324559.wav'),
 ('neutrality', 'neutrality_40830219533825574.wav'),
 ('anger', 'anger_9994360410126384.wav')]

In [236]:
files_dict = dict()

In [237]:
for elem in tuples2:
    if elem[0] in files_dict:
        files_dict[elem[0]].append(elem[1])
    else:
        files_dict[elem[0]] = [elem[1]]

In [239]:
files_dict['anger']

['anger_9992815572033784.wav',
 'anger_9995287435466227.wav',
 'anger_9994545645155642.wav',
 'anger_9994360410126384.wav']

In [283]:
np.random.randint(0,4)

2